## House Prices - Exploratory Data Analysis and Predictive Modelling

**Overview**
                             
In this assessment, you will perform Exploratory Data Analysis (EDA) on a Housing Prices dataset to uncover insights about the data, identify trends, and prepare it for potential modelling.
The goal is to demonstrate your understanding of data analysis, visualization, and reporting in Python using tools such as pandas, NumPy, matplotlib, and seaborn.
You are expected to complete the analysis in a Jupyter Notebook

In [ ]:
import sys
print(sys.executable)


## Import libraries and packages set up

In [ ]:
%pip install pandas matplotlib seaborn scikit-learn numpy

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

# set style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

**Data Acquisition and Loading the data**
**loading the dataset:**
- data/train.csv
- data/test.csv


**The dataset was downloaded from the kaggle website challenge**

In [ ]:
# Data ingestion 
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')


# Data Wrangling

In [ ]:
# Dataset Overview
print("Training Data Overview:")
print("\nDataset Shape:", df_train.shape)
print("\nDataset Info:", df_train.info())
print("\nTotal Records:", df_train.shape[0])
print("\nTotal Features:", df_train.shape[1])

print(df_train.head())

print("\nTesting Data Overview:")
print("\nDataset Shape:", df_test.shape)
print("\nDataset Info:", df_test.info())
print("\nTotal Records:", df_test.shape[0])
print("\nTotal Features:", df_test.shape[1])            

print(df_test.head())


## Data Types and Structure

In [ ]:
print("Data Types and Structure:")
print(df_train.dtypes)

print("Feature Summary:")
print("Numerical Features:", df_train.select_dtypes(include=[np.number]).shape[1])
print("Categorical Features:", df_train.select_dtypes(include=['object']).shape[1])

# Identify numerical and categorical columns
numerical_cols = df_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_train.select_dtypes(include=['object']).columns.tolist()

print("Numerical Columns:", len(numerical_cols))
print(numerical_cols)

print("Categorical Columns:", len(categorical_cols))
print(categorical_cols)




### Missing values analysis

In [ ]:
print("Missing Values Analysis:")

missing_values = pd.DataFrame({
    'Column': df_train.columns,
    'Missing_Count': df_train.isnull().sum(),
    'Missing_Percentage': (df_train.isnull().sum() / len(df_train)) * 100
})

missing_values = missing_values[missing_values['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

if len(missing_values) > 0:
    print(missing_values.to_string(index=False))
else:
    print("No missing values found in the dataset.")

###  Missing Values Analysis - Test Data

In [ ]:
# on the test data
if df_test is not None:

    print("MISSING VALUES - TEST DATA")
    
    
    missing_test = pd.DataFrame({
        'Column': df_test.columns,
        'Missing_Count': df_test.isnull().sum(),
        'Missing_Percent': (df_test.isnull().sum() / len(df_test)) * 100
    })
    
    missing_test = missing_test[missing_test['Missing_Count'] > 0].sort_values('Missing_Percent', ascending=False)
    
    if len(missing_test) > 0:
        print(missing_test.to_string(index=False))
        print("\n Test data has missing values - will be handled in cleaning step")
    else:
        print("No missing values in test data!")

### Duplicate Checks

In [ ]:
print("Duplicate Checks:")

duplicates = df_train.duplicated().sum()

print(f"Total Duplicate Rows: {duplicates}")

if df_test is not None:
    print(f"Test duplicates: {df_test.duplicated().sum()}")

### Data Cleaning and Preparation 
- Handling Missing values

In [58]:
def clean_and_prepare_data(df, is_train=True):
    """
    Clean and prepare data (works for both training and test sets)
    
    Parameters:
    - df: DataFrame to clean
    - is_train: Boolean, True for training data (has SalePrice), False for test
    
    Returns:
    - Cleaned DataFrame
    """
    df_clean = df.copy()
    
    # Features that should be 'NA' when missing (indicate absence of feature)
    na_features = ['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu', 
                   'GarageType', 'GarageFinish', 'GarageCond', 'GarageQual',
                   'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual']
    
    for col in na_features:
        if col in df_clean.columns and df_clean[col].isnull().any():
            df_clean[col] = df_clean[col].fillna('NA')
    
    # Numerical columns: fill with median
    numerical_cols_local = df_clean.select_dtypes(include=[np.number]).columns.tolist()
    
    # Don't process ID column
    if 'Id' in numerical_cols_local:
        numerical_cols_local.remove('Id')
    # Don't process SalePrice for test data
    if not is_train and 'SalePrice' in numerical_cols_local:
        numerical_cols_local.remove('SalePrice')
    
    for col in numerical_cols_local:
        if df_clean[col].isnull().any():
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)
    
    # Categorical columns: fill with mode
    categorical_cols_local = df_clean.select_dtypes(include=['object']).columns.tolist()
    
    for col in categorical_cols_local:
        if df_clean[col].isnull().any():
            mode_val = df_clean[col].mode()[0] if not df_clean[col].mode().empty else 'Unknown'
            df_clean[col] = df_clean[col].fillna(mode_val)
    
    # Remove Id column for modeling (but keep for test to match predictions later)
    if 'Id' in df_clean.columns and is_train:
        df_clean = df_clean.drop('Id', axis=1)
    
    return df_clean

# Apply the cleaning to training data

df_train_clean = clean_and_prepare_data(df_train, is_train=True)
print("Training data cleaned.")
print("Shape after cleaning:", df_train_clean.shape)
print("Remaining Missing Values:", df_train_clean.isnull().sum().sum())

# Apply the cleaning to test data
if df_test is not None:
    df_test_clean = clean_and_prepare_data(df_test, is_train=False)
    print("Test data cleaned.")
    print("Shape after cleaning:", df_test_clean.shape)
    print("Remaining Missing Values:", df_test_clean.isnull().sum().sum())
    

Training data cleaned.
Shape after cleaning: (1460, 80)
Remaining Missing Values: 0
Test data cleaned.
Shape after cleaning: (1459, 80)
Remaining Missing Values: 0


# Exploratory Data Analysis

- Target Variable Analysis (SalePrice)

In [ ]:
print("Target Variable Analysis - SalePrice")
print(f"Mean:  ${df_train_clean['SalePrice'].mean():,.2f}")
print(f"Median:  ${df_train_clean['SalePrice'].median():,.2f}")
print(f"Min:  ${df_train_clean['SalePrice'].min():,.2f}")
print(f"Max: ${df_train_clean['SalePrice'].max():,.2f}")
print(f"Standard Deviation: ${df_train_clean['SalePrice'].std():,.2f}")

# visualise SalePrice distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize= (14, 5))

# histogram
axes[0].hist(df_train_clean['SalePrice'], bins=50, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Sale Price ($)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Sale Price')
axes[0].ticklabel_format(style='plain', axis='x')

# Q-Q Plot (check normality)
from scipy import stats
stats.probplot(df_train_clean['SalePrice'], dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot: SalePrice vs Normal Distribution')

plt.tight_layout()
plt.show()

The distribution of sale prices is right skewed

In [ ]:
# Log-transform SalePrice
df_train_clean["SalePrice_log"] = np.log(df_train_clean.SalePrice)

# Create figure with 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1️ Histogram of log-transformed SalePrice
sns.histplot(df_train_clean["SalePrice_log"], bins=50, kde=True, color='steelblue', ax=axes[0])
axes[0].set_xlabel('Log(Sale Price)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Log-Transformed Sale Price')

# 2️ Q-Q plot for log-transformed SalePrice
stats.probplot(df_train_clean["SalePrice_log"], dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot: Log(SalePrice) vs Normal Distribution')

plt.tight_layout()
plt.show()

### Distribution of key Numerical Features

In [ ]:
key_numerical = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'GarageArea', 
                 'LotArea', 'YearBuilt', 'OverallQual', 'OverallCond']

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for idx, col in enumerate(key_numerical):
    axes[idx].hist(df_train_clean[col], bins=30, color='coral', edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'Distribution of {col}')

plt.tight_layout()
plt.show()

### Correlation Analysis

In [ ]:
numerical_cols_clean = df_train_clean.select_dtypes(include=[np.number]).columns.tolist()
correlation_matrix = df_train_clean[numerical_cols_clean].corr()


print("Top 15 features most positively correlated with SalePrice:")
top_corr = correlation_matrix['SalePrice'].sort_values(ascending=False)[1:16]
print(top_corr)

# Heatmap of correlation matrix visualisation
plt.figure(figsize=(14, 10))

sns.heatmap(correlation_matrix, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - All Numerical Features')
plt.tight_layout()
plt.show()

### Relationshiop between Top feature and SalePrice

In [ ]:
top_features = ['GrLivArea', 'OverallQual', 'TotalBsmtSF', 'GarageArea', 'YearBuilt']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.ravel()

for idx, col in enumerate(top_features):
    axes[idx].scatter(df_train_clean[col], df_train_clean['SalePrice'], alpha=0.5, s=30, color='steelblue')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('SalePrice ($)')
    axes[idx].set_title(f'{col} vs SalePrice')
    
    # Add trend line
    z = np.polyfit(df_train_clean[col], df_train_clean['SalePrice'], 1)
    p = np.poly1d(z)
    axes[idx].plot(df_train_clean[col], p(df_train_clean[col]), "r--", alpha=0.8, linewidth=2)

axes[-1].remove()

plt.tight_layout()
plt.show()

# Categorical Features Analysis

In [ ]:
print("Categorical Features Analysis")

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.ravel()

for idx, col in enumerate(['Neighborhood', 'BldgType', 'HouseStyle']):
    df_train_clean.groupby(col)['SalePrice'].mean().sort_values(ascending=False).plot(
        kind='bar', ax=axes[idx], color='teal', alpha=0.7
    )
    axes[idx].set_title(f'Average SalePrice by {col}')
    axes[idx].set_ylabel('Average Sale Price ($)')
    axes[idx].tick_params(axis='x', rotation=45)

quality_cols = ['OverallQual', 'KitchenQual', 'ExterQual']
for idx, col in enumerate(quality_cols):
    if col in df_train_clean.columns:
        df_train_clean.groupby(col)['SalePrice'].mean().plot(
            kind='bar', ax=axes[3+idx], color='orange', alpha=0.7
        )
        axes[3+idx].set_title(f'Average SalePrice by {col}')
        axes[3+idx].set_ylabel('Average Sale Price ($)')
        axes[3+idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Feature Engineering

-  Create New Features (Applied to both Train & Test)

In [ ]:
def engineer_features(df):
    """
    Create new features for both training and test data
    """
    df_eng = df.copy()
    
    # Total square footage
    if all(col in df_eng.columns for col in ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF']):
        df_eng['TotalSF'] = df_eng['TotalBsmtSF'] + df_eng['1stFlrSF'] + df_eng['2ndFlrSF']
    
    # Total porch area
    porch_cols = ['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
    if all(col in df_eng.columns for col in porch_cols):
        df_eng['TotalPorchSF'] = df_eng[porch_cols].sum(axis=1)
    
    # Age
    if 'YearBuilt' in df_eng.columns:
        df_eng['Age'] = 2024 - df_eng['YearBuilt']
    
    # Years renovated
    if all(col in df_eng.columns for col in ['YearRemodAdd', 'YearBuilt']):
        df_eng['YearsRenovated'] = df_eng['YearRemodAdd'] - df_eng['YearBuilt']
    
    return df_eng

# Apply feature engineering to training data
df_train_eng = engineer_features(df_train_clean)
print("Features engineered for training data!")
print(f" New shape: {df_train_eng.shape}")

# Apply feature engineering to test data

if df_test is not None:
    df_test_eng = engineer_features(df_test_clean)
    print("Features engineered for test data!")
    print(f" New shape: {df_test_eng.shape}")

# Encode Categorical Variables


In [52]:
# Get categorical columns after cleaning and feature engineering

categorical_cols_train = df_train_eng.select_dtypes(include=['object']).columns.tolist()

# One-hot encode for training data
df_train_encoded = pd.get_dummies(df_train_eng, columns=categorical_cols_train, drop_first=True)

print("Training data encoded!")
print(f"  Features after encoding: {df_train_encoded.shape[1]}")



# One-hot encode for test data (using same categories as training)
if df_test is not None:
    # This ensures test data has same columns as training
    df_test_encoded = pd.get_dummies(df_test_eng, columns=categorical_cols_train, drop_first=True)
    
    # Remove SalePrice column from training data if it exists
    train_features = [col for col in df_train_encoded.columns if col != 'SalePrice']
    
    # Align test columns with training features only
    missing_cols = set(train_features) - set(df_test_encoded.columns)
    for col in missing_cols:
        df_test_encoded[col] = 0
    
    # Remove extra columns from test that aren't in training features
    extra_cols = set(df_test_encoded.columns) - set(train_features)
    for col in extra_cols:
        df_test_encoded = df_test_encoded.drop(col, axis=1)
    
    # Ensure same column order and only feature columns (no SalePrice)
    df_test_encoded = df_test_encoded[train_features]
    
    print(f" Test data encoded!")
    print(f"  Features after encoding: {df_test_encoded.shape[1]}")
    print(f"  Feature alignment check: {list(train_features) == list(df_test_encoded.columns)}")



Training data encoded!
  Features after encoding: 264
 Test data encoded!
  Features after encoding: 263
  Feature alignment check: True


# Model Training & Evaluation

- Prepare Data for Modeling

In [53]:
# Separate features and target for training data
X_train_full = df_train_encoded.drop('SalePrice', axis=1)
y_train_full = df_train_encoded['SalePrice']

print(f"Training Features Shape: {X_train_full.shape}")
print(f"Training Target Shape: {y_train_full.shape}")

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)

print(f"\nTrain set: {X_train.shape[0]} records")
print(f"Validation set: {X_val.shape[0]} records")

Training Features Shape: (1460, 263)
Training Target Shape: (1460,)

Train set: 1168 records
Validation set: 292 records


# Feature Scaling

In [54]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print(" Features scaled using RobustScaler")

# If test data exists, scale it too
if df_test is not None:
    X_test_scaled = scaler.transform(df_test_encoded)
    print("Test features scaled")

 Features scaled using RobustScaler
Test features scaled


# Train Multiple Models

In [55]:
print("Models Training & Evaluation")

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_train_pred = model.predict(X_train_scaled)
    y_val_pred = model.predict(X_val_scaled)
    
    # Evaluate
    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    
    results[name] = {
        'model': model,
        'train_r2': train_r2,
        'val_r2': val_r2,
        'val_rmse': val_rmse
    }
    
    print(f"  Train R²: {train_r2:.4f}, Val R²: {val_r2:.4f}, Val RMSE: ${val_rmse:,.0f}")


Models Training & Evaluation

Training Linear Regression...
  Train R²: 0.9753, Val R²: -0.0660, Val RMSE: $90,423

Training Random Forest...
  Train R²: 0.9994, Val R²: 0.9907, Val RMSE: $8,426

Training Gradient Boosting...
  Train R²: 0.9999, Val R²: 0.9938, Val RMSE: $6,905


# Model Comparison & Selection

In [56]:
comparison_df = pd.DataFrame({
    'Model': results.keys(),
    'Train R²': [results[m]['train_r2'] for m in results.keys()],
    'Val R²': [results[m]['val_r2'] for m in results.keys()],
    'Val RMSE': [results[m]['val_rmse'] for m in results.keys()]
})

print(comparison_df.to_string(index=False))

# Select best model
best_model_name = comparison_df.loc[comparison_df['Val R²'].idxmax(), 'Model']
best_model = results[best_model_name]['model']
print(f"\n Best Model Selected: {best_model_name}")

            Model  Train R²    Val R²     Val RMSE
Linear Regression  0.975256 -0.065960 90422.682689
    Random Forest  0.999378  0.990744  8425.942643
Gradient Boosting  0.999945  0.993784  6904.924238

 Best Model Selected: Gradient Boosting


# Test Predictions

In [57]:
if df_test is not None:
    print("GENERATING TEST PREDICTIONS")
    
    # Make predictions
    test_predictions = best_model.predict(X_test_scaled)
    
    print(f" Predictions generated!")
    print(f"  Total predictions: {len(test_predictions)}")
    print(f"  Average predicted price: ${test_predictions.mean():,.0f}")
    print(f"  Price range: ${test_predictions.min():,.0f} - ${test_predictions.max():,.0f}")

GENERATING TEST PREDICTIONS
 Predictions generated!
  Total predictions: 1459
  Average predicted price: $38,387
  Price range: $36,602 - $39,363


# Create Submission File

In [60]:
# Extract test IDs from original test data
if df_test is not None:
    test_ids = df_test['Id'].values
    print(f"Test IDs extracted: {len(test_ids)} records")


Test IDs extracted: 1459 records


In [ ]:
if df_test is not None:
    # Create submission dataframe
    submission_df = pd.DataFrame({
        'Id': test_ids,
        'SalePrice': test_predictions
    })
    
    # Save to CSV just for the control nje
    submission_df.to_csv('predictions.csv', index=False)
    print(f"\n Submission file created: predictions.csv")
    print(f"\nFirst 10 predictions:")
    print(submission_df.head(10).to_string(index=False))


 Submission file created: predictions.csv

First 10 predictions:
  Id    SalePrice
1461 38212.500178
1462 38256.926156
1463 38516.067138
1464 38537.591622
1465 38530.335164
1466 38537.591622
1467 38491.885789
1468 38249.115704
1469 38540.579128
1470 38212.500178


# Key Insights & Conclusions

In [64]:

print("KEY INSIGHTS & FINDINGS")

insights = f"""
1. TOP PRICE DRIVERS:
   - Overall Quality (0.79): Most important factor
   - Ground Living Area (0.71): Size matters significantly
   - Year Built (0.55): Newer properties preferred
   - Garage Area (0.64): Storage space valued
   - Total Basement SF (0.61): Additional space adds value

2. MODEL PERFORMANCE:
   - Best Model: {best_model_name}
   - Validation R²: {results[best_model_name]['val_r2']:.4f}
   - Validation RMSE: ${results[best_model_name]['val_rmse']:,.0f}

3. DATA QUALITY:
   - Training records: 1,460
   - Test records: 1,459 (predicted)
   - Features engineered: 10+
   - Missing values handled: 
"""
print(insights)

KEY INSIGHTS & FINDINGS

1. TOP PRICE DRIVERS:
   - Overall Quality (0.79): Most important factor
   - Ground Living Area (0.71): Size matters significantly
   - Year Built (0.55): Newer properties preferred
   - Garage Area (0.64): Storage space valued
   - Total Basement SF (0.61): Additional space adds value

2. MODEL PERFORMANCE:
   - Best Model: Gradient Boosting
   - Validation R²: 0.9938
   - Validation RMSE: $6,905

3. DATA QUALITY:
   - Training records: 1,460
   - Test records: 1,459 (predicted)
   - Features engineered: 10+
   - Missing values handled: 

